# Automatic Test Generation

In this project, we will build an automatic test generation and grading platform! All we have to do is to provide a topic, the number of questions and the number of options for each question! Based on this information, a suitable test is generated, 
presented to the user and graded automatically!

## Library Imports

In [1]:
import os
import openai

## Get the OpenAI API Key

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

## Telling GPT how to generate the test

We tell GPT to create a multiple choiz quiz. Hence we define the topic, the number of possible answers as well as the number of questions. To enable automatical grading later, GPT needs to incorporate the correct answer!

In [2]:
# We will define a function now
def create_test_prompt(topic, num_questions, num_possible_answers):
    prompt = f"Create a multiple choice quiz on the topic of {topic} consisting of {num_questions} questions. " \
    + f"Each question should have {num_possible_answers} options. "\
    + f"Also include the correct answer for each question using the starting string 'Correct Answer : '."
    return prompt   

In [3]:
# Now we will test the prompt creation
print(create_test_prompt("Python", 4, 4))

Create a multiple choice quiz on the topic of Python consisting of 4 questions. Each question should have 4 options. Also include the correct answer for each question using the starting string 'Correct Answer : '.


## Make the OpenAI API Call

In [4]:
response = openai.Completion.create(engine='text-davinci-003',
                                    prompt = create_test_prompt("Python", 4, 4),
                                    max_tokens = 256,
                                    temperature = 0.7)

# Check the response now
response

<OpenAIObject text_completion id=cmpl-856pvZHXVH99Qg715Xq0MIqlnrE9Y at 0x7f8092236c00> JSON: {
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-856pvZHXVH99Qg715Xq0MIqlnrE9Y",
  "object": "text_completion",
  "created": 1696228611,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n\nQ1. What is the name of a popular programming language?\nA. Java \nB. Python \nC. C++ \nD. JavaScript\nCorrect Answer : B. Python \n\nQ2. What is the purpose of a for loop in Python?\nA. To store data \nB. To iterate over a sequence \nC. To create a function \nD. To define a variable\nCorrect Answer : B. To iterate over a sequence \n\nQ3. What is a string in Python?\nA. A sequence of characters \nB. A data type \nC. A collection of data \nD. A loop\nCorrect Answer : A. A sequence of characters \n\nQ4. What is the correct syntax to define a function in Pyth

In [5]:
# Parse the actual response prompt
response['choices'][0]['text']

'\n\nQ1. What is the name of a popular programming language?\nA. Java \nB. Python \nC. C++ \nD. JavaScript\nCorrect Answer : B. Python \n\nQ2. What is the purpose of a for loop in Python?\nA. To store data \nB. To iterate over a sequence \nC. To create a function \nD. To define a variable\nCorrect Answer : B. To iterate over a sequence \n\nQ3. What is a string in Python?\nA. A sequence of characters \nB. A data type \nC. A collection of data \nD. A loop\nCorrect Answer : A. A sequence of characters \n\nQ4. What is the correct syntax to define a function in Python?\nA. def function_name \nB. function_name() \nC. function function_name \nD. create function_name\nCorrect Answer : A. def function_name'

## Q/A Extraction

We now need to extract the questions and answers to present them to the students later

In [13]:
def create_student_view(test, num_questions):
    student_view = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer :"):
            student_view[question_number] += line+"\n"
        else:

            if question_number < num_questions:
                question_number+=1
                student_view[question_number] = ""
    return student_view

In [14]:
# test the function
print(create_student_view(response['choices'][0]['text'], 4))

{1: '\n\nQ1. What is the name of a popular programming language?\nA. Java \nB. Python \nC. C++ \nD. JavaScript\n', 2: '\nQ2. What is the purpose of a for loop in Python?\nA. To store data \nB. To iterate over a sequence \nC. To create a function \nD. To define a variable\n', 3: '\nQ3. What is a string in Python?\nA. A sequence of characters \nB. A data type \nC. A collection of data \nD. A loop\n', 4: '\nQ4. What is the correct syntax to define a function in Python?\nA. def function_name \nB. function_name() \nC. function function_name \nD. create function_name\n'}


In [15]:
# Now a function to extract the answers
def extract_answers(test, num_questions):
    answers = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if line.startswith("Correct Answer :"):
            answers[question_number] += line+"\n"

            if question_number < num_questions:
                question_number+=1
                answers[question_number] = ""
    return answers

In [16]:
# test the function
print(extract_answers(response['choices'][0]['text'], 4))

{1: 'Correct Answer : B. Python \n', 2: 'Correct Answer : B. To iterate over a sequence \n', 3: 'Correct Answer : A. A sequence of characters \n', 4: 'Correct Answer : A. def function_name\n'}


## Exam Simulation

Based on extracted questions, we can now simulate the exam

In [25]:
def take(student_view):
    answers = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        answers[question] = answer
    return answers

In [26]:
student_answers = take(create_student_view(response["choices"][0]["text"], 4))



Q1. What is the name of a popular programming language?
A. Java 
B. Python 
C. C++ 
D. JavaScript


Q2. What is the purpose of a for loop in Python?
A. To store data 
B. To iterate over a sequence 
C. To create a function 
D. To define a variable


Q3. What is a string in Python?
A. A sequence of characters 
B. A data type 
C. A collection of data 
D. A loop


Q4. What is the correct syntax to define a function in Python?
A. def function_name 
B. function_name() 
C. function function_name 
D. create function_name



## Automatic Grading 

Based on the student's answers and correct answers, we can now grade the test!

In [27]:
def grade(correct_answer_dict, answers):
    correct_answers = 0
    for question, answer in answers.items():
        if answer.upper() == correct_answer_dict[question].upper()[16]:
            correct_answers+=1
    grade = 100 * correct_answers / len(answers)

    if grade < 60:
        passed = "Not passed!"
    else:
        passed = "Passed!"
    return f"{correct_answers} out of {len(answers)} correct! You achieved: {grade} % : {passed}"

In [28]:
grade(extract_answers(response["choices"][0]["text"], 4), student_answers)

'0 out of 4 correct! You achieved: 0.0 % : Not passed!'